#BERTopic on Quran Urdu Translations
In this notebook I have implemented Topic Modeling on Urdu translations based topic modelling technique BERTopic.

Shaista Zulfiqar

## Mounting Google Drive
If the dataset is on Google Drive then you have to mount over google drive with collaboratory.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive




#Installing required dependencies
**One thing to remember is that after installing libraries you have to restart the run time again so that other dependencies are not affected by it.**

In [ ]:
!pip install bertopic
!pip install -U sentence-transformers
!pip install urduhack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB

In [ ]:
!pip install --upgrade keras tensorflow-addons


# Importing required dependencies
We will import numpy, pandas and re, bertopic, gensim library for now. other libraries will be imported in the notebook later.

Pandas will be used to create a Dataframe and handle the csv file. Numpy will be used for the faster computation of arrays to save time. re library will be used for the cleaning of data. gensim library is used to get coherence score and train LDA. bertopic is used to train bertopic on our Quran-UTM dataset with using pretrained language models Multilingial MiniLM

In [ ]:
import pandas as pd
import numpy as np
import re
from bertopic import BERTopic
from urduhack.normalization import remove_diacritics
from gensim.models import LdaMulticore
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from gensim.models.coherencemodel import CoherenceModel
import gensim.corpora as corpora
#optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

##DataFrame




In [ ]:

junapd = pd.read_table("Set your data path here", header=None, encoding='utf-8')

print(junapd.head(5))

stopspd=pd.read_csv('/content/drive/MyDrive/stopwords.txt',names=['List']) #your path

stopspd

                                                   0
0    1|1|اللہ کے نام سے شروع جو بہت مہربان رحمت والا
1  1|2|سب خوبیاں اللہ کو جو مالک سارے جہان والوں کا،
2                          1|3|بہت مہربان رحمت والا،
3                               1|4|روز جزا کا مالک،
4        1|5|ہم تجھی کو پوجیں اور تجھی سے مدد چاہیں،


,List
0,کی
1,ہیں
2,ہے
3,رہا
4,رہی
...,...
396,گئی
397,ہونے
398,وجہ
399,ہوگیا


## Cleaning of Data
After collecting the eight Quran Urdu translations, we observed these translation had some irrelevant information, such as metadata, punctuation, and diacritics. So we cleaned the translations for topic modeling

Stopwords are common words that are often filtered out during text processing in natural language processing (NLP) tasks. These words are considered to have little or no value in conveying the actual meaning of the text. We take list of 401 stopwords for topic modelling. Stopwords are removed in post preprocessing phase

In [ ]:
import re
#Remove Urdu Punctuation
def remove_urdu_punctuation(text):
    pattern = r'[؛؟،٫٬‘’“”«»!"٪&\'\*\+,-./:;<=>؟@^_`()[]{|}~]'

    cleaned_text = re.sub(pattern, '', text)

    return cleaned_text


In [ ]:
# Remove trailing metadata lines
index_to_drop = junapd[junapd[0].str.startswith("#")].index
junapd.drop(index_to_drop, inplace=True)

# Remove verse reference from every line
junapd[0] = junapd[0].str.replace(r'\d+\|\d+\|', '', regex=True)

# Remove diacritics
junapd[0] = junapd[0].apply(remove_diacritics)

# Remove punctuation
junapd[0] = junapd[0].apply(remove_urdu_punctuation)

# Reset index
junapd.reset_index(drop=True, inplace=True)

# Display the cleaned DataFrame
print("Cleaned DataFrame:")
print(junapd.head())



Cleaned DataFrame:
                                               0
0    اللہ کے نام سے شروع جو بہت مہربان رحمت والا
1  سب خوبیاں اللہ کو جو مالک سارے جہان والوں کا،
2                          بہت مہربان رحمت والا،
3                               روز جزا کا مالک،
4        ہم تجھی کو پوجیں اور تجھی سے مدد چاہیں،


In [ ]:
def remove_nonbreaking_space(text):
    return re.sub(r'\xa0', ' ', text)

junapd[0] = junapd[0].apply(remove_nonbreaking_space)

In [ ]:
# Save the cleaned DataFrame to a text file
file_path = 'cleaned_Kanzuliman_text.txt'
junapd[0].to_csv(file_path, sep='\n', index=False, header=False, encoding='utf-8')

print(f'Text has been saved to {file_path}')

Text has been saved to cleaned_Kanzuliman_text.txt


In [ ]:
# Convert the DataFrame to a list of strings
data = junapd[0].tolist()

In [ ]:
print(len(data))

6236


In [ ]:
print(data[56:90])

['اور جب ہم نے تمہارے لئے دریا پھاڑ دیا تو تمہیں بچالیا اور فرعون والوں کو تمہاری آنکھوں کے سامنے ڈبو دیا -', 'اور جب ہم نے موسی سے چالیس رات کا وعدہ فرمایا پھر اس کے پیچھے تم نے بچھڑے کی پوجا شروع کردی اور تم ظالم تھے -', 'پھر اس کے بعد ہم نے تمہیں معافی دی کہ کہیں تم احسان مانو -', 'اور جب ہم نے موسی کو کتاب عطا کی اور حق و باطل میں تمیز کردینا کہ کہیں تم راہ آؤ -', 'اور جب موسی نے اپنی قوم سے کہا اے میری قوم تم نے بچھڑا بناکر اپنی جانوں پر ظلم کیا تو اپنے پیدا کرنے والے کی طرف رجوع لاؤ تو آپس میں ایک دوسرے کو قتل کردو یہ تمہارے پیدا کرنے والے کے نزدیک تمہارے لیے بہتر ہے تو اس نے تمہاری توبہ قبول کی بیشک وہی ہے بہت توبہ قبول کرنے والا مہربان -', 'اور جب تم نے کہا اے موسی ہم ہرگز تمہارا یقین نہ لائیں گے جب تک اعلانیہ خدا کو نہ دیکھ لیں تو تمہیں کڑک نے آلیا اور تم دیکھ رہے تھے ۔', 'پھر مرے پیچھے ہم نے تمہیں زندہ کیا کہ کہیں تم احسان مانو-', 'اور ہم نے ابر کو تمہارا سائبان کیا اور تم پر من اور سلوی اتارا کھاؤ ہماری دی ہوئی ستھری چیزیں اور انہوں نے کچھ ہمارا نہ بگاڑا ہاں اپنی ہی جانوں کو

# BERTopic Training
The default  bertopic embedding model is paraphrase-multilingual-MiniLM-L12-v2 when selecting language="multilingual". We take MiniLM Model from [sentence-tranformer](https://www.sbert.net/docs/pretrained_models.html) and create custom document embedding and passed it to the bertopic model for training.

In [ ]:
#create custom embedding
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(data, show_progress_bar=True)
print(embeddings)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/195 [00:00<?, ?it/s]

[[-0.1029857   0.6000064  -0.27553585 ...  0.01412378  0.34318107
   0.09316683]
 [ 0.00934135  0.5204709  -0.18125957 ... -0.10896025  0.09319583
   0.02816286]
 [-0.06815726  0.13377076  0.10087793 ...  0.00802712  0.2562524
   0.10564535]
 ...
 [-0.0074717   0.1903338  -0.10379498 ... -0.22501028  0.22306737
   0.06328312]
 [-0.07753874 -0.06097173 -0.10399013 ...  0.02249005  0.211614
  -0.03487154]
 [ 0.01718189  0.16830286 -0.05594749 ... -0.13294442 -0.02409285
   0.03989932]]


In [ ]:
stop_words_list = stopspd['List'].tolist()

In [ ]:
#pass vectorizer_model to bertopic for stopwords removal
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(stop_words= stop_words_list)


In [ ]:
#UMAP for dimention reduction
from umap import UMAP
dim_model = UMAP(n_components=4, random_state=42)

In [ ]:
# # #KMeans used for clustering
from sklearn.cluster import KMeans

cluster_model = KMeans(n_clusters=5, random_state=42)

In [ ]:
np.random.seed(42)

In [ ]:
topic_model = BERTopic(language="urdu", low_memory=True ,calculate_probabilities=True,  top_n_words=10,hdbscan_model=cluster_model, umap_model=dim_model, verbose=True, vectorizer_model=vectorizer_model)

In [ ]:
#Fit documents in bertopic
topics, probs = topic_model.fit_transform(data,embeddings)

2024-06-13 07:48:43,001 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-06-13 07:49:39,563 - BERTopic - Dimensionality - Completed ✓
2024-06-13 07:49:39,566 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-06-13 07:49:39,751 - BERTopic - Cluster - Completed ✓
2024-06-13 07:49:39,764 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-06-13 07:49:39,966 - BERTopic - Representation - Completed ✓


In [ ]:
print(probs)

None


In [ ]:
#topics that assign to each document
print(topics)

[1, 1, 2, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 1, 1, 1, 1, 0, 0, 2, 2, 1, 0, 1, 0, 1, 1, 2, 1, 1, 0, 0, 1, 1, 2, 2, 2, 1, 0, 2, 2, 2, 2, 1, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 3, 3, 3, 0, 3, 3, 1, 0, 0, 2, 2, 3, 1, 1, 0, 1, 0, 0, 3, 2, 2, 1, 0, 2, 1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 0, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 2, 0, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 2, 2, 1, 1, 2, 2, 0, 1, 0, 0, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 0, 1, 1, 1, 1, 0, 1, 2, 1, 2, 1, 1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 1, 0, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 2, 2, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 0, 1, 2, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 

In [ ]:
topic_model.get_topic_freq()

,Topic,Count
2,0,2476
0,1,1892
1,2,1593
3,3,243
4,4,32


In [ ]:
document_topics = topic_model.get_topics()

In [ ]:
#topics with score
print(document_topics)

{0: [('بیشک', 0.04881132841952738), ('دن', 0.04743224655188579), ('تمہیں', 0.03821504236942546), ('کوئی', 0.029555293646861974), ('تمہارے', 0.027988326523352036), ('پاس', 0.0262144197319727), ('زمین', 0.02547865878422397), ('عذاب', 0.023274854184489636), ('ضرور', 0.022373810330171995), ('بات', 0.02222204299495381)], 1: [('اللہ', 0.1508064261701155), ('بیشک', 0.0436911860874506), ('تمہیں', 0.038239269014892774), ('تمہارے', 0.03617091641121193), ('ایمان', 0.03589091566026826), ('والا', 0.03554709527897281), ('کوئی', 0.033090942117558024), ('اے', 0.029036004352267834), ('راہ', 0.027875151930312276), ('رسول', 0.026099921637856544)], 2: [('رب', 0.09598748046310816), ('بیشک', 0.05590942930146236), ('اے', 0.03973307306330769), ('ایمان', 0.03972026335334262), ('والا', 0.038227037019175555), ('تمہارے', 0.036340219434785846), ('پاس', 0.029528777311606275), ('میرے', 0.028923722344149427), ('تمہیں', 0.026378600558120604), ('عذاب', 0.025706790386094847)], 3: [('موسی', 0.21849518606300297), ('فرعون'

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,2476,0_بیشک_دن_تمہیں_کوئی,"[بیشک, دن, تمہیں, کوئی, تمہارے, پاس, زمین, عذا...",[اور وہی ہے جس نے تمہیں زندہ کیا اور پھر تمہیں...
1,1,1892,1_اللہ_بیشک_تمہیں_تمہارے,"[اللہ, بیشک, تمہیں, تمہارے, ایمان, والا, کوئی,...","[تم فرماؤ وہ اللہ ہے وہ ایک ہے, اے رسول پہنچا ..."
2,2,1593,2_رب_بیشک_اے_ایمان,"[رب, بیشک, اے, ایمان, والا, تمہارے, پاس, میرے,...",[عرض کی، اے میرے رب! میں نے اپنی جان پر زیادتی...
3,3,243,3_موسی_فرعون_قوم_بیشک,"[موسی, فرعون, قوم, بیشک, نوح, اے, اسرائیل, بنی...","[اور تو نے اپنی قوم سے کیوں جلدی کی اے موسی, ا..."
4,4,32,4_جھٹلاؤ_کونسی_نعمت_رب,"[جھٹلاؤ, کونسی, نعمت, رب, دونوں, سی, کون, انس,...","[تو اپنے رب کی کونسی نعمت جھٹلاؤ گے،, تو اپنے ..."


In [ ]:
topic_distr, _ = topic_model.approximate_distribution(data, window=3, min_similarity=0.01)

100%|██████████| 7/7 [00:09<00:00,  1.31s/it]


In [ ]:
print(topic_distr)

[[0.04955011 0.47536868 0.42313415 0.05194706 0.        ]
 [0.08767806 0.59588361 0.1996907  0.11674764 0.        ]
 [0.0719931  0.33034273 0.49887421 0.09878996 0.        ]
 ...
 [0.43345964 0.23952317 0.23119315 0.09582404 0.        ]
 [0.25848013 0.38446768 0.22640452 0.13064768 0.        ]
 [0.64087029 0.14418115 0.21494856 0.         0.        ]]


In [ ]:
topic_model.visualize_distribution(topic_distr[0], width=600,height=600, title="Topic Probability Distribution of Kanzuliman Translation")

# Evaluation
we used three evaluation metrics to compare the results.

1. The coherence score is used to capture the degree of similarity between the words within each topic, with higher scores indicating more coherent topics. We used two coherence metrics NPMI and Cv Score.
2. IRBO measures are used to assess how different and distinct the topics are in a topic model.


### Coherence Score
To evaluate the model topics coherence we use [Gensim](https://radimrehurek.com/gensim/models/coherencemodel.html) library

In [ ]:
topic_model.update_topics(data, vectorizer_model=vectorizer_model)

In [ ]:
texts = [[word for word in str(document).split() if word not in stop_words_list] for document in data] #if word not in stop_words_list
id2word = corpora.Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
topics_bert=[]
for i in topic_model.get_topics():
  row=[]
  topic= topic_model.get_topic(i)
  for word in topic:
     row.append(word[0])
  topics_bert.append(row)

In [ ]:
print(topics_bert)

[['بیشک', 'دن', 'تمہیں', 'کوئی', 'تمہارے', 'پاس', 'زمین', 'عذاب', 'ضرور', 'بات'], ['اللہ', 'بیشک', 'تمہیں', 'تمہارے', 'ایمان', 'والا', 'کوئی', 'اے', 'راہ', 'رسول'], ['رب', 'بیشک', 'اے', 'ایمان', 'والا', 'تمہارے', 'پاس', 'میرے', 'تمہیں', 'عذاب'], ['موسی', 'فرعون', 'قوم', 'بیشک', 'نوح', 'اے', 'اسرائیل', 'بنی', 'فرمایا', 'رب'], ['جھٹلاؤ', 'کونسی', 'نعمت', 'رب', 'دونوں', 'سی', 'کون', 'انس', 'نعمتوں', 'اے']]


In [ ]:
# compute Coherence Score CV

cm = CoherenceModel(topics=topics_bert, texts=texts, dictionary=id2word, coherence='c_v')
coherence = round(cm.get_coherence(),2)
print('\nCV Score: ', coherence)


CV Score:  0.51


In [ ]:
# compute Coherence Score NPMI

cm = CoherenceModel(topics=topics_bert, texts=texts, dictionary=id2word, coherence='c_npmi')
coherence = round(cm.get_coherence(),2)
print('\nNPMI Score: ', coherence)


NPMI Score:  0.02


**Diversity Score**

upload rbo.py file before importing

In [ ]:
import itertools
from rbo import rbo
import numpy as np

class InvertedRBO:
    def __init__(self):
        pass

    def irbo(self, topics, topk=10, weight=0.9):
        """
        Calculate inverted Rank Biased Overlap (RBO) as a measure of topic diversity from a list of lists of words.

        :param topics: A list of lists of words representing different topics.
        :param topk: The number of top words on which RBO will be computed.
        :param weight: Weight of each agreement at depth d: p**(d-1). When set to 1.0, there is no weight,
                       and the RBO returns to average overlap.
        :return: The inverted RBO topic diversity score.
        """
        if topk <= 0:
            raise ValueError("topk must be a positive integer.")

        num_topics = len(topics)
        if num_topics == 0:
            raise ValueError("topics list cannot be empty.")

        if topk > len(topics[0]):
            raise Exception('Words in topics are less than topk')

        collect = []
        for list1, list2 in itertools.combinations(topics, 2):
            rbo_val = rbo(list1[:topk], list2[:topk], p=weight)[2]
            collect.append(rbo_val)

        Irbo_score = 1 - np.mean(collect)
        return Irbo_score

In [ ]:
inverted_rbo_calculator = InvertedRBO()
IRBO= round(inverted_rbo_calculator.irbo(topics_bert, topk=10, weight=0.9),2)
print("Inverted RBO Score:", IRBO)

Inverted RBO Score: 0.8


# Visualize Topics

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart(n_words=10,width=220, height=270, title="Topic Word Scores of Kanzuliman Translation")

# Model serialization

In [ ]:
# Save model
topic_model.save("my_model")

2024-06-09 10:52:31,148 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.
/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [ ]:
loaded_topic_model = topic_model.load("my_model")
loaded_topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,2476,0_بیشک_دن_تمہیں_کوئی,"[بیشک, دن, تمہیں, کوئی, تمہارے, پاس, زمین, عذا...",[اور وہی ہے جس نے تمہیں زندہ کیا اور پھر تمہیں...
1,1,1892,1_اللہ_بیشک_تمہیں_تمہارے,"[اللہ, بیشک, تمہیں, تمہارے, ایمان, والا, کوئی,...","[تم فرماؤ وہ اللہ ہے وہ ایک ہے, اے رسول پہنچا ..."
2,2,1593,2_رب_بیشک_اے_ایمان,"[رب, بیشک, اے, ایمان, والا, تمہارے, پاس, میرے,...",[عرض کی، اے میرے رب! میں نے اپنی جان پر زیادتی...
3,3,243,3_موسی_فرعون_قوم_بیشک,"[موسی, فرعون, قوم, بیشک, نوح, اے, اسرائیل, بنی...","[اور تو نے اپنی قوم سے کیوں جلدی کی اے موسی, ا..."
4,4,32,4_جھٹلاؤ_کونسی_نعمت_رب,"[جھٹلاؤ, کونسی, نعمت, رب, دونوں, سی, کون, انس,...","[تو اپنے رب کی کونسی نعمت جھٹلاؤ گے،, تو اپنے ..."


# LDA

We use the [ parallelized Latent Dirichlet Allocation (LDA)](https://radimrehurek.com/gensim/models/ldamulticore.html) from Gensim.

Note: for LDA you have to define topics number in advance.

In [ ]:
n_topics=5
lda = LdaMulticore(corpus, id2word=id2word, random_state=42, num_topics=5)
topics = lda.show_topics(num_topics=5, formatted=False)

#Extract the words from the topics
topics_list = []
for _, topic_words in topics:
    words = [word for word, _ in topic_words]
    topics_list.append(words)

print(topics_list)

In [ ]:
#CV Score
cm = CoherenceModel(topics=topics_list, texts=texts, corpus=corpus, dictionary=id2word, coherence='c_v')
coherence_lda = round(cm.get_coherence(),2)
print('\nCV Score: ', coherence_lda)


CV Score:  0.51


In [ ]:
#NPMI Score
cm = CoherenceModel(topics=topics_list, texts=texts, corpus=corpus, dictionary=id2word, coherence='c_npmi')
coherence_lda = round(cm.get_coherence(),2)
print('\nNPMI Score: ', coherence_lda)


NPMI Score:  -0.0


In [ ]:
# Calculate IRBO Score
inverted_rbo_calculator = InvertedRBO()
IRBO_LDA = round(inverted_rbo_calculator.irbo(topics_list, topk=10, weight=0.9),2)
print('\nIRBO Score: ',IRBO_LDA)


IRBO Score:  0.54


#NMF
We use Gensim library for implementation of NMF

Note: for NMF you have to define topics number in advance.

In [ ]:
#Using Gensim
from gensim import corpora, models


# Train NMF model
num_topics = 5  # Define the number of topics
nmf_model = models.Nmf(corpus, num_topics=num_topics, id2word=id2word, random_state=42)

# Extract topics
topics = []
for topic_id in range(num_topics):
    topic_words = nmf_model.show_topic(topic_id, topn=10)
    topic_words = [word for word, _ in topic_words]
    topics.append(topic_words)

# Print topics in the desired format
print(topics)

In [ ]:
#Calculate CV Score
cm = CoherenceModel(topics=topics, texts=texts, corpus=corpus, dictionary=id2word, coherence='c_v')
coherence_nmf = round(cm.get_coherence(),2)
print('\nCV Score: ', coherence_nmf)

In [ ]:
#Calculate NPMI Score
cm = CoherenceModel(topics=topics, texts=texts, corpus=corpus, dictionary=id2word, coherence='c_npmi')
coherence_nmf = round(cm.get_coherence(),2)
print('\nNPMI Score: ', coherence_nmf)

In [ ]:
# Calculate IRBO Score
inverted_rbo_calculator = InvertedRBO()
IRBO_NMF = round(inverted_rbo_calculator.irbo(topics, topk=10, weight=0.9),2)c
print('\nIRBO Score: ',IRBO_NMF)